<a href="https://colab.research.google.com/github/thaisbeham/Applied_deep_learning/blob/main/Assigment2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM - Fake News detection

Fake - 0
Real - 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import pandas as pd
#from utils import merge_datasets, smaller_set
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import string
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



Read data

In [17]:
def read_news(path, column_text_name):
  news = pd.read_csv(path)
  news[column_text_name] = news[column_text_name].astype(str)
  return news

def merge_split_datasets(real, fake, n_testsize = 0.15):
    #take unique values from real data
    #real = real.drop_duplicates(subset = ['sentence'])

    # take 13000 rows from real data
    #real = real.sample(n = 13000, random_state= 2, axis = 0)

    # add column label
    real['label'] = 1
    fake['label'] = 0

    real = real[['text', 'label']]
    fake = fake[['text', 'label']]
    #real.rename(columns = {'sentence': 'text'}, inplace = True)

    #merge real and fake
    # use only the sentence column on fake
    merged = pd.DataFrame(real.append(fake, ignore_index = True))
    # shuffle the rows
    merged = merged.sample(frac = 1, random_state= 1, ignore_index = True)#.reset_index()

    train, test = train_test_split(merged, test_size = n_testsize, random_state = 1)

    X_test = test['text']
    Y_test = test['label']

    #create validation set
    X = train['text']
    Y = train['label'].astype(int)    
    X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size=0.15, random_state=42)

    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test





In [18]:
fake_news1 = read_news("/content/drive/MyDrive/dados_/fake.csv", "text")
fake_news2 = read_news("/content/drive/MyDrive/dados_/Fake_bisaillon.csv", "text")

real_news1 = read_news("/content/drive/MyDrive/dados_/df_2016.csv", "sentence")
real_news1.rename(columns = {'sentence': 'text'}, inplace = True) 
real_news2 = read_news("/content/drive/MyDrive/dados_/True_bisaillon.csv", "text")

#shape of each dataset
print("fake_news1: ", np.shape(fake_news1), "\nfake_news2: ",  np.shape(fake_news2), "\nreal_news1: ",  np.shape(real_news1),"\nreal_news2: ", np.shape(real_news2)  )

real_news1 = real_news1.sample(n = 15000, random_state= 2, axis = 0)


# separate the location of the news from the news text (e.g. "WASHINGTON (Reuters)")
real_news2[["loc", "text"]] = real_news2['text'].str.split('-',1, expand = True)
real_news2.head()





fake_news1:  (12999, 20) 
fake_news2:  (23481, 4) 
real_news1:  (105606, 3) 
real_news2:  (21417, 4)


,title,text,subject,date,loc
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fi...,politicsNews,"December 29, 2017",WASHINGTON (Reuters)
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links be...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos to...,politicsNews,"December 30, 2017",WASHINGTON (Reuters)
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Pos...,politicsNews,"December 29, 2017",SEATTLE/WASHINGTON (Reuters)


In [30]:

real_news = pd.DataFrame(real_news1["text"].append(real_news2["text"], ignore_index = True))

fake_news = pd.DataFrame(fake_news1["text"].append(fake_news2["text"], ignore_index = True))


real_news = real_news.dropna()


In [31]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news, fake_news)

In [32]:
#parameters tunning
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 10

#tokenizer
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [33]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [34]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [38]:
model.fit(sequences_matrix,Y_train,batch_size=n_batchsize,epochs=n_epochs,
          validation_split=0.2)

Epoch 1/10
330/330 [==============================] - 5s 17ms/step - loss: 0.2249 - accuracy: 0.9141 - val_loss: 0.1531 - val_accuracy: 0.9449
Epoch 2/10
330/330 [==============================] - 5s 15ms/step - loss: 0.1458 - accuracy: 0.9487 - val_loss: 0.1732 - val_accuracy: 0.9385
Epoch 3/10
330/330 [==============================] - 8s 24ms/step - loss: 0.1292 - accuracy: 0.9536 - val_loss: 0.1528 - val_accuracy: 0.9491
Epoch 4/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1188 - accuracy: 0.9578 - val_loss: 0.1336 - val_accuracy: 0.9524
Epoch 5/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1092 - accuracy: 0.9607 - val_loss: 0.1791 - val_accuracy: 0.9422
Epoch 6/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1005 - accuracy: 0.9640 - val_loss: 0.1628 - val_accuracy: 0.9472
Epoch 7/10
330/330 [==============================] - 5s 15ms/step - loss: 0.0945 - accuracy: 0.9671 - val_loss: 0.1339 - val_accuracy: 0.9514

In [39]:
def predict_result(X_test, Y_test):
  test_sequences = tok.texts_to_sequences(X_test)
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  accr = model.evaluate(test_sequences_matrix,Y_test)
  print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
  predict_labels = model.predict(test_sequences_matrix) 

  # convert the range predicition into 0s or 1s
  Y_pred =[ 1 if i>0.5 else 0 for i in predict_labels  ] 

  print("Number Real news:",sum(Y_pred))
  print("Number Fake news:",len(Y_pred)-sum(Y_pred))
  print()
  print("Classification Report")
  print(classification_report(Y_test, Y_pred, target_names = ['class 0', 'class 1']))
  print()
  print("Confusion Matrix")
  matrix=confusion_matrix(Y_test,Y_pred,labels=[0,1])
  cm=pd.DataFrame(matrix,index=['class_0 pred','class_1 pred'],columns=['class_0 True','class_1 True'])
  print(cm)



In [40]:
# predict on validation set

predict_result(X_valid, Y_valid)

291/291 [==============================] - 2s 5ms/step - loss: 0.1453 - accuracy: 0.9522
Test set
  Loss: 0.145
  Accuracy: 0.952
291/291 [==============================] - 2s 4ms/step
Number Real news: 4943
Number Fake news: 4351

Classification Report
              precision    recall  f1-score   support

     class 0       0.98      0.93      0.95      4593
     class 1       0.93      0.98      0.95      4701

    accuracy                           0.95      9294
   macro avg       0.95      0.95      0.95      9294
weighted avg       0.95      0.95      0.95      9294


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          4250           343
class_1 pred           101          4600


In [41]:
# predict on test set

predict_result(X_test, Y_test)


342/342 [==============================] - 2s 5ms/step - loss: 0.1424 - accuracy: 0.9504
Test set
  Loss: 0.142
  Accuracy: 0.950
342/342 [==============================] - 1s 4ms/step
Number Real news: 5773
Number Fake news: 5162

Classification Report
              precision    recall  f1-score   support

     class 0       0.97      0.93      0.95      5406
     class 1       0.93      0.97      0.95      5529

    accuracy                           0.95     10935
   macro avg       0.95      0.95      0.95     10935
weighted avg       0.95      0.95      0.95     10935


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          5013           393
class_1 pred           149          5380
